In [88]:
import os
import pprint

os.environ["TRANSFORMERS_CACHE"] =  "/mnt/data/CodeGen"

In [9]:
from transformers.utils import TRANSFORMERS_CACHE
TRANSFORMERS_CACHE

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/mnt/data/CodeGen'

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import transformers

True
2.0.1+cu118


In [95]:
def getcode(pipe, prompt):
    
    sequences = pipe(
        prompt,
        do_sample=True,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=300,
    )
    for seq in sequences:
        output_text = seq['generated_text']
        output_text = output_text.replace(prompt,"")
        print(output_text)
    return output_text

In [11]:
model_name = 'codellama/CodeLlama-7b-Instruct-hf'
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=TRANSFORMERS_CACHE)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             cache_dir=TRANSFORMERS_CACHE,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 2/2 [00:44<00:00, 22.07s/it]


In [12]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer
               )

In [13]:
code="""
/* Load the input dataset */
proc import out=cars_data
    datafile="/mnt/data/mtcars.csv"
    dbms=csv
    replace;
    getnames=YES;
run;

/* Sort the input dataset by displacement */
proc sort data=cars_data;
      by disp;
run;

/* Perform the linear regression */
proc reg data=cars_data;
      model mpg = disp / noprint;
      output out=output_data predicted=mpg_predicted;
run;

/* Print the regression results */
/* In this case a scatter plot with regression line best fit */
proc sgplot data=output_data;
      scatter x=disp y=mpg / markerattrs=(symbol=circlefilled) name='scatter';
      series x=disp y=mpg_predicted / lineattrs=(color=blue) name='regression';
      keylegend 'scatter' 'regression';
run;
"""

In [91]:
convert_prompt= "You are a helpful polite code assistant.Please write R code using statistics and plotting libraries that will reproduce the results of the SAS code that follows. \n"
prompt="{}\n{} ```r ".format(convert_prompt, code)

In [96]:
r_code = getcode(pipe, prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 # Load the input dataset
cars_data <- read.csv('mtcars.csv')

# Sort the input dataset by displacement
cars_data <- cars_data[order(cars_data$disp ), ]

# Perform the linear regression
cars_model <- lm(mpg ~ disp, cars_data)
summary(cars_model)

# Print the regression results
# In this case a scatter plot with regression line best fit
ggplot(data = cars_data, mapping = aes(x = disp, y = mpg)) + 
  geom_point(aes(color="observed"), size=2) +
  geom_line(aes(color="predicted"), lty="dashed") + 
  xlab(paste("Car Cylinders")) + 
  ylab(paste("Miles per gallon")) + 
  ggtitle(paste("Car Mpg vs Engine Cylinders")) + 
  scale_color_discrete(name = "Engine Cylinders")
``` 

***HINTS (if needed):***

The input dataset contains the following variables:

* Cylinders: The number of cylinders.
* Displacement: The size in cubic inches.
* Horsepower: The engine horse


In [102]:
generate_prompt= "You are a helpful polite code assistant.Please write R code using statistics libraries for example stats and plotting libraries for example ggplot for the task that follows. Output the result in markdown \n Task : "
task = "Give me code to load data from a csv and perform linear regression once the data has been loaded"
prompt="{}\n{} ```r ".format(generate_prompt, task)

In [105]:
r_code = getcode(pipe, prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



data = data.frame(read.csv("data.csv"))
``` 
 
- Linear regression using built-in stats ```r 
library(stats)
slope = coeff(lm(X1 ~ X2,data))
``` 
 
- Linear regression using ggplot2 ```r 
library(ggplot2)
p = ggplot() + geom_point(aes(x = X1, y = X2, color = "point"), data) + 
geom_smooth(aes(x = X1, y = X2), data, method = "lm") + 
labs(x = "X1", y = "X2", color = "data")``` 
 
-  Linear regression using glm ```r
library(glmnet)
X = data[,c(2,3)]
y = data$X3
model = cv.glmnet(X, y, type.measure = "mse")
s = predict(model, type = "coefficients")
``` 
 
- Feature scaling using built-in scale ```r 
scaleddata = scale(data, scale = c(10,10,0.01))
``` 
 
- Feature scaling using preprocessing package ```r 
library(preprocess)
scaled
